In [1]:
!pip install https://github.com/explosion/spacy-models/releases/download/en_core_web_lg-2.2.5/en_core_web_lg-2.2.5.tar.gz

     |████████████████████████████████| 827.9 MB 5.4 kB/s 
  Created wheel for en-core-web-lg: filename=en_core_web_lg-2.2.5-py3-none-any.whl size=829180942 sha256=dc5767aa5a74722e73888d20867ce340d56caa1732c76a59a53d26067e1a1e03
  Stored in directory: /root/.cache/pip/wheels/11/95/ba/2c36cc368c0bd339b44a791c2c1881a1fb714b78c29a4cb8f5
Successfully built en-core-web-lg


In [6]:
!pip install bert-tensorflow

     |████████████████████████████████| 64 kB 2.1 MB/s 


In [1]:
#%cd ..
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
!ln -s /content/gdrive/MyDrive/ /mydrive
!ls /mydrive

'2020 slides'
 2021-online-language-offline-impact-main
 An_Xception_Based_Convolutional_Neural_Network_for_Scene_Image_Classification_with_Transfer_Learning.pdf
'API key'
'Application for Lease- Form for Email (Applicant) (1).gdoc'
'Application for Lease- Form for Email (Applicant).gdoc'
'Application for Lease- Form for Email (Applicant).pdf'
'Colab Notebooks'
'CV -Xizhi Wu.pdf'
'FOW Documents'
 MyDrive
'NT15 LEASE 2021.gdoc'
'NT15 LEASE 2021.pdf'
'Review Group 9.gdoc'
 Xizhi_Wu_UnderGrad_Transcript.pdf
 教授资格证身份证.pdf


In [3]:
%cd gdrive/MyDrive/2021-online-language-offline-impact-main

/content/gdrive/MyDrive/2021-online-language-offline-impact-main


In [4]:
!ls

 a0529-a0611_UK		       dataframe_result_csv_UK
 a0529-a0611_US		       Github
 ccc_compiled.csv	       heatmap529_UK.ipynb
 csv			       heatmap530.ipynb
 csv.zip		       Heat_Map_Online_language.ipynb
 dataframe_csv		       __MACOSX
 dataframe_csv_EU	       mapping_UK
 dataframe_csv_UK	       mapping_US
 dataframe_result_529_UK.csv   non_violant1.csv
 dataframe_result_530_UK.csv   non_violant2.csv
 dataframe_result_531_UK.csv   UK
 dataframe_result_csv	       violant.csv
 dataframe_result_csv_EU      '新建 Microsoft PowerPoint 演示文稿.pptx'


In [7]:
import os
import collections
import ast
import pandas as pd
import numpy as np
from scipy import stats
from datetime import datetime, timedelta
import spacy
import tensorflow as tf
import tensorflow_hub as hub
from tensorflow.keras import layers
from bert import tokenization
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
import matplotlib.pyplot as plt

In [8]:
ct_path_601 = 'UK/06_05.csv'

In [9]:
ct = pd.read_csv(ct_path_601)

In [10]:
ct.shape[1]

47

In [11]:
columns1=['platformId', 'platform', 'date', 'updated', 'type1', 'title', 'caption', 'description', 'message','expandedLinks', 'link', 'postUrl', 'subscriberCount', 'score', 'media_type', 'media_numl', 'actual_likeCount', 'actual_shareCount', 'actual_commentCount', 'actual_loveCount', 'actual_wowCount', 'actual_hahaCount', 'actual_sadCount', 'actual_angryCount', 'actual_thankfulCount', 'actual_careCount', 'expected_likeCount', 'expected_shareCount', 'expected_commentCount', 'expected_loveCount', 'expected_wowCount', 'expected_hahaCount', 'expected_sadCount', 'expected_angryCount', 'expected_thankfulCount', 'expected_careCount', 'account_id', 'account_name', 'account_handle', 'account_profileImage', 'account_accountType', 'account_pageAdminTopCountry', 'account_pageDescription', 'account_verified', 'account_pageCreatedDate', 'languageCode', 'legacyId', 'post_id']
columns2=['platformId', 'platform', 'date', 'updated', 'type1', 'title', 'caption', 'description', 'message', 'expandedLinks', 'link', 'postUrl', 'subscriberCount', 'score', 'media', 'actual_likeCount', 'actual_shareCount', 'actual_commentCount', 'actual_loveCount', 'actual_wowCount', 'actual_hahaCount', 'actual_sadCount', 'actual_angryCount', 'actual_thankfulCount', 'actual_careCount', 'expected_likeCount', 'expected_shareCount', 'expected_commentCount', 'expected_loveCount', 'expected_wowCount', 'expected_hahaCount', 'expected_sadCount', 'expected_angryCount', 'expected_thankfulCount', 'expected_careCount', 'account_id', 'account_name', 'account_handle', 'account_profileImage', 'account_accountType', 'account_pageAdminTopCountry', 'account_pageDescription', 'account_verified', 'account_pageCreatedDate', 'languageCode', 'legacyId', 'post_id']

In [12]:
#len(columns1)
len(columns2)

47

In [13]:
ct.columns=columns2

In [14]:
ct

,platformId,platform,date,updated,type1,title,caption,description,message,expandedLinks,link,postUrl,subscriberCount,score,media,actual_likeCount,actual_shareCount,actual_commentCount,actual_loveCount,actual_wowCount,actual_hahaCount,actual_sadCount,actual_angryCount,actual_thankfulCount,actual_careCount,expected_likeCount,expected_shareCount,expected_commentCount,expected_loveCount,expected_wowCount,expected_hahaCount,expected_sadCount,expected_angryCount,expected_thankfulCount,expected_careCount,account_id,account_name,account_handle,account_profileImage,account_accountType,account_pageAdminTopCountry,account_pageDescription,account_verified,account_pageCreatedDate,languageCode,legacyId,post_id
0,48597654813_10158381384639814,Facebook,2020-06-05 23:59:48,2021-08-31 05:39:29,link,Black Lives Matter protest in Ada honors Antho...,kxii.com,-,Hundreds of people came together for a Black L...,1.0,https://www.kxii.com/content/news/Black-Lives-...,https://www.facebook.com/48597654813/posts/101...,125613,-2.015625,1.0,25,11,7,16,0,3,0,2,0,0,24,14,35,5,8,14,16,13,0,4,24653,KXII-TV,channel12,https://scontent-sea1-1.xx.fbcdn.net/v/t1.6435...,facebook_page,US,Texoma's Breaking News & Weather Authority ser...,True,2009-02-04 21:03:03,en,0,24653|10158381384639814
1,57406800343_10163759708125344,Facebook,2020-06-05 23:59:38,2021-07-27 09:14:41,status,Black Lives Matter protest in Ada honors Antho...,kxii.com,-,Check out the latest episode of the In the Zon...,1.0,https://www.kxii.com/content/news/Black-Lives-...,https://www.facebook.com/groups/57406800343/pe...,1373,-0.666667,1.0,0,1,0,0,0,0,0,0,0,1,1,1,0,1,0,0,0,0,0,0,8520182,Intertwine Chicago,channel12,https://scontent-sea1-1.xx.fbcdn.net/v/t1.1816...,facebook_group,US,Texoma's Breaking News & Weather Authority ser...,False,2009-02-04 21:03:03,en,0,8520182|10163759708125344
2,102051553222110_2992368327523737,Facebook,2020-06-05 23:59:37,2021-07-08 21:08:55,photo,Black Lives Matter protest in Ada honors Antho...,kxii.com,"Today, No Peace Until Justice marches are taki...",Read Linda Silas' full statement here: https:/...,2.0,https://www.facebook.com/Linda4Workers/photos/...,https://www.facebook.com/102051553222110/posts...,8927,2.681818,1.0,44,10,0,5,0,0,0,0,0,0,7,5,2,3,1,1,2,1,0,2,190096,Canadian Federation of Nurses Unions (CFNU),NursesUnions,https://scontent-sea1-1.xx.fbcdn.net/v/t1.6435...,facebook_page,CA,The official Facebook page of the Canadian Fed...,False,2011-06-01 18:46:23,en,0,190096|2992368327523737
3,322314941151928_3245155888867804,Facebook,2020-06-05 23:59:32,2021-08-24 19:47:41,status,Black Lives Matter protest in Ada honors Antho...,kxii.com,"Today, No Peace Until Justice marches are taki...","Family, friends, neighbors; We wanted to take ...",2.0,https://www.facebook.com/Linda4Workers/photos/...,https://www.facebook.com/322314941151928/posts...,2926,5.915888,1.0,201,208,33,190,1,0,0,0,0,28,52,23,8,10,2,1,10,1,0,8,2904250,The Coeur d'Alene Tribe,cdatribe,https://scontent-sea1-1.xx.fbcdn.net/v/t31.181...,facebook_page,US,Welcome to the official page for the Schitsu'u...,False,2012-02-08 21:52:13,en,0,2904250|3245155888867804
4,146265275458081_3032237670194146,Facebook,2020-06-05 23:59:26,2021-06-28 20:32:20,status,Black Lives Matter protest in Ada honors Antho...,kxii.com,"Today, No Peace Until Justice marches are taki...",Check out the latest episode of the In the Zon...,2.0,https://www.facebook.com/Linda4Workers/photos/...,https://www.facebook.com/groups/14626527545808...,1188,-22.000000,1.0,0,0,0,0,0,0,0,0,0,0,2,2,3,2,0,2,0,0,0,1,10227323,Y.O.U. Young Opulent & Unique,cdatribe,https://scontent-sea1-1.xx.fbcdn.net/v/t1.1816...,facebook_group,US,Welcome to the official page for the Schitsu'u...,False,2012-02-08 21:52:13,en,0,10227323|3032237670194146
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35409,460670040801275_14339257

In [15]:
ct['message'] = ct['message'].fillna('')
ct['description'] = ct['description'].fillna('')
ct['title'] = ct['title'].fillna('')

In [16]:
nlp = spacy.load('en_core_web_lg', disable=["tok2vec", "tagger", "parser", "attribute_ruler", "lemmatizer"])

In [17]:
def ner(txt):
    ner = dict()

    doc = nlp(txt)
    for ent in doc.ents:
        if ent.label_ == 'GPE':
            if ent.text not in ner:
                ner[ent.text] = 1
            else:
                ner[ent.text] += 1
    return ner

In [18]:
def ner_inline(doc):
    ner = dict()

    for ent in doc.ents:
        if ent.label_ == 'GPE':
            if ent.text not in ner:
                ner[ent.text] = 1
            else:
                ner[ent.text] += 1
    return ner

In [19]:
def ner_pipe(texts):
    batch = []
    for doc in nlp.pipe(texts, batch_size=200, n_threads=16):
        batch.append(ner_inline(doc))
    return batch

In [20]:
ct['NER-msg'] = ner_pipe(ct['message'])

In [21]:
ct['NER-msg'] = ct['message'].apply(ner)

In [22]:
ct['NER-desc'] = ct['description'].apply(ner)

In [23]:
ct['NER-title'] = ct['title'].apply(ner)

In [24]:
ct[['NER-msg', 'NER-desc']].head()

,NER-msg,NER-desc
0,{'Ada': 1},{}
1,{},{}
2,{},{'Canada': 2}
3,{'Minneapolis': 2},{'Canada': 2}
4,{},{'Canada': 2}


In [25]:
ct.to_csv('a0605.csv')

In [26]:
ct=pd.read_csv('a0605.csv',encoding='ISO-8859-1', lineterminator='\n')

In [27]:
!ls

 a0529-a0611_UK		       dataframe_result_csv_UK
 a0529-a0611_US		       Github
 a0605.csv		       heatmap529_UK.ipynb
 ccc_compiled.csv	       heatmap530.ipynb
 csv			       Heat_Map_Online_language.ipynb
 csv.zip		       __MACOSX
 dataframe_csv		       mapping_UK
 dataframe_csv_EU	       mapping_US
 dataframe_csv_UK	       non_violant1.csv
 dataframe_result_529_UK.csv   non_violant2.csv
 dataframe_result_530_UK.csv   UK
 dataframe_result_531_UK.csv   violant.csv
 dataframe_result_csv	      '新建 Microsoft PowerPoint 演示文稿.pptx'
 dataframe_result_csv_EU


In [28]:
###
#ct=pd.read_csv('a0601.csv')

In [29]:
ct

,Unnamed: 0,platformId,platform,date,updated,type1,title,caption,description,message,expandedLinks,link,postUrl,subscriberCount,score,media,actual_likeCount,actual_shareCount,actual_commentCount,actual_loveCount,actual_wowCount,actual_hahaCount,actual_sadCount,actual_angryCount,actual_thankfulCount,actual_careCount,expected_likeCount,expected_shareCount,expected_commentCount,expected_loveCount,expected_wowCount,expected_hahaCount,expected_sadCount,expected_angryCount,expected_thankfulCount,expected_careCount,account_id,account_name,account_handle,account_profileImage,account_accountType,account_pageAdminTopCountry,account_pageDescription,account_verified,account_pageCreatedDate,languageCode,legacyId,post_id,NER-msg,NER-desc,NER-title
0,0,48597654813_10158381384639814,Facebook,2020-06-05 23:59:48,2021-08-31 05:39:29,link,Black Lives Matter protest in Ada honors Antho...,kxii.com,-,Hundreds of people came together for a Black L...,1.0,https://www.kxii.com/content/news/Black-Lives-...,https://www.facebook.com/48597654813/posts/101...,125613,-2.015625,1.0,25,11,7,16,0,3,0,2,0,0,24,14,35,5,8,14,16,13,0,4,24653,KXII-TV,channel12,https://scontent-sea1-1.xx.fbcdn.net/v/t1.6435...,facebook_page,US,Texoma's Breaking News & Weather Authority ser...,True,2009-02-04 21:03:03,en,0,24653|10158381384639814,{'Ada': 1},{},{}
1,1,57406800343_10163759708125344,Facebook,2020-06-05 23:59:38,2021-07-27 09:14:41,status,Black Lives Matter protest in Ada honors Antho...,kxii.com,-,Check out the latest episode of the In the Zon...,1.0,https://www.kxii.com/content/news/Black-Lives-...,https://www.facebook.com/groups/57406800343/pe...,1373,-0.666667,1.0,0,1,0,0,0,0,0,0,0,1,1,1,0,1,0,0,0,0,0,0,8520182,Intertwine Chicago,channel12,https://scontent-sea1-1.xx.fbcdn.net/v/t1.1816...,facebook_group,US,Texoma's Breaking News & Weather Authority ser...,False,2009-02-04 21:03:03,en,0,8520182|10163759708125344,{},{},{}
2,2,102051553222110_2992368327523737,Facebook,2020-06-05 23:59:37,2021-07-08 21:08:55,photo,Black Lives Matter protest in Ada honors Antho...,kxii.com,"Today, No Peace Until Justice marches are taki...",Read Linda Silas' full statement here: https:/...,2.0,https://www.facebook.com/Linda4Workers/photos/...,https://www.facebook.com/102051553222110/posts...,8927,2.681818,1.0,44,10,0,5,0,0,0,0,0,0,7,5,2,3,1,1,2,1,0,2,190096,Canadian Federation of Nurses Unions (CFNU),NursesUnions,https://scontent-sea1-1.xx.fbcdn.net/v/t1.6435...,facebook_page,CA,The official Facebook page of the Canadian Fed...,False,2011-06-01 18:46:23,en,0,190096|2992368327523737,{},{'Canada': 2},{}
3,3,322314941151928_3245155888867804,Facebook,2020-06-05 23:59:32,2021-08-24 19:47:41,status,Black Lives Matter protest in Ada honors Antho...,kxii.com,"Today, No Peace Until Justice marches are taki...","Family, friends, neighbors; We wanted to take ...",2.0,https://www.facebook.com/Linda4Workers/photos/...,https://www.facebook.com/322314941151928/posts...,2926,5.915888,1.0,201,208,33,190,1,0,0,0,0,28,52,23,8,10,2,1,10,1,0,8,2904250,The Coeur d'Alene Tribe,cdatribe,https://scontent-sea1-1.xx.fbcdn.net/v/t31.181...,facebook_page,US,Welcome to the official page for the Schitsu'u...,False,2012-02-08 21:52:13,en,0,2904250|3245155888867804,{'Minneapolis': 2},{'Canada': 2},{}
4,4,146265275458081_3032237670194146,Facebook,2020-06-05 23:59:26,2021-06-28 20:32:20,status,Black Lives Matter protest in Ada honors Antho...,kxii.com,"Today, No Peace Until Justice marches are taki...",Check out the latest episode of the In the Zon...,2.0,https://www.facebook.com/Linda4Workers/photos/...,https://www.facebook.com/groups/14626527545808...,1188,-22.000000,1.0,0,0,0,0,0,0,0,0,0,0,2,2,3,2,0,2,0,0,0,1,10227323,Y.O.U. Young Opulent & Unique,cdatribe,https://scontent-sea1-1.xx.fbcdn.net/v/t1.1816...,facebook_group,US,Welcome to the official page for the Schitsu'u...,False,2012-02-08 21:52:13,en,0,10227323|3032237670194146,{},{'Canada': 2},{}
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,

In [30]:
ct['NER-msg'] = ct['NER-msg'].apply(ast.literal_eval)
ct['NER-desc'] = ct['NER-desc'].apply(ast.literal_eval)
ct['NER-title'] = ct['NER-title'].apply(ast.literal_eval)

In [31]:
#ct_ner = ct[((ct['NER-msg'] != {}) | (ct['NER-desc'] != {}) | (ct['NER-title'] != {}))]

In [32]:
###
ct_ner = ct

In [33]:
ct_ner[['NER-msg', 'NER-desc', 'NER-title']].head()

,NER-msg,NER-desc,NER-title
0,{'Ada': 1},{},{}
1,{},{},{}
2,{},{'Canada': 2},{}
3,{'Minneapolis': 2},{'Canada': 2},{}
4,{},{'Canada': 2},{}


In [34]:
ct_ner['NER-msg'] = ct_ner['NER-msg'].apply(lambda x: list(set(x.keys())))
ct_ner['NER-desc'] = ct_ner['NER-desc'].apply(lambda x: list(set(x.keys())))
ct_ner['NER-title'] = ct_ner['NER-title'].apply(lambda x: list(set(x.keys())))

In [35]:
ct_ner['NER-msg']

0                     [Ada]
1                        []
2                        []
3             [Minneapolis]
4                        []
                ...        
35409                    []
35410                    []
35411                    []
35412    [Lebanon, America]
35413                    []
Name: NER-msg, Length: 35414, dtype: object

In [36]:
ct_ner['NER'] = ct_ner['NER-msg'] + ct_ner['NER-desc'] + ct_ner['NER-title']

In [37]:
ct_ner['NER']

0                                          [Ada]
1                                             []
2                                       [Canada]
3                          [Minneapolis, Canada]
4                                       [Canada]
                          ...                   
35409                                         []
35410                                         []
35411    [IOWA CITY, Iowa, Iowa City, Iowa City]
35412       [Lebanon, America, Lebanon, America]
35413                         [Lebanon, America]
Name: NER, Length: 35414, dtype: object

In [38]:
ct_ner['NER'] = ct_ner['NER'].apply(set)
ct_ner['NER'] = ct_ner['NER'].apply(list)

In [39]:
ct_ner['NER'] = ct_ner['NER'].apply(lambda x: [s.lower() for s in x])

In [40]:
ct_ner

,Unnamed: 0,platformId,platform,date,updated,type1,title,caption,description,message,expandedLinks,link,postUrl,subscriberCount,score,media,actual_likeCount,actual_shareCount,actual_commentCount,actual_loveCount,actual_wowCount,actual_hahaCount,actual_sadCount,actual_angryCount,actual_thankfulCount,actual_careCount,expected_likeCount,expected_shareCount,expected_commentCount,expected_loveCount,expected_wowCount,expected_hahaCount,expected_sadCount,expected_angryCount,expected_thankfulCount,expected_careCount,account_id,account_name,account_handle,account_profileImage,account_accountType,account_pageAdminTopCountry,account_pageDescription,account_verified,account_pageCreatedDate,languageCode,legacyId,post_id,NER-msg,NER-desc,NER-title,NER
0,0,48597654813_10158381384639814,Facebook,2020-06-05 23:59:48,2021-08-31 05:39:29,link,Black Lives Matter protest in Ada honors Antho...,kxii.com,-,Hundreds of people came together for a Black L...,1.0,https://www.kxii.com/content/news/Black-Lives-...,https://www.facebook.com/48597654813/posts/101...,125613,-2.015625,1.0,25,11,7,16,0,3,0,2,0,0,24,14,35,5,8,14,16,13,0,4,24653,KXII-TV,channel12,https://scontent-sea1-1.xx.fbcdn.net/v/t1.6435...,facebook_page,US,Texoma's Breaking News & Weather Authority ser...,True,2009-02-04 21:03:03,en,0,24653|10158381384639814,[Ada],[],[],[ada]
1,1,57406800343_10163759708125344,Facebook,2020-06-05 23:59:38,2021-07-27 09:14:41,status,Black Lives Matter protest in Ada honors Antho...,kxii.com,-,Check out the latest episode of the In the Zon...,1.0,https://www.kxii.com/content/news/Black-Lives-...,https://www.facebook.com/groups/57406800343/pe...,1373,-0.666667,1.0,0,1,0,0,0,0,0,0,0,1,1,1,0,1,0,0,0,0,0,0,8520182,Intertwine Chicago,channel12,https://scontent-sea1-1.xx.fbcdn.net/v/t1.1816...,facebook_group,US,Texoma's Breaking News & Weather Authority ser...,False,2009-02-04 21:03:03,en,0,8520182|10163759708125344,[],[],[],[]
2,2,102051553222110_2992368327523737,Facebook,2020-06-05 23:59:37,2021-07-08 21:08:55,photo,Black Lives Matter protest in Ada honors Antho...,kxii.com,"Today, No Peace Until Justice marches are taki...",Read Linda Silas' full statement here: https:/...,2.0,https://www.facebook.com/Linda4Workers/photos/...,https://www.facebook.com/102051553222110/posts...,8927,2.681818,1.0,44,10,0,5,0,0,0,0,0,0,7,5,2,3,1,1,2,1,0,2,190096,Canadian Federation of Nurses Unions (CFNU),NursesUnions,https://scontent-sea1-1.xx.fbcdn.net/v/t1.6435...,facebook_page,CA,The official Facebook page of the Canadian Fed...,False,2011-06-01 18:46:23,en,0,190096|2992368327523737,[],[Canada],[],[canada]
3,3,322314941151928_3245155888867804,Facebook,2020-06-05 23:59:32,2021-08-24 19:47:41,status,Black Lives Matter protest in Ada honors Antho...,kxii.com,"Today, No Peace Until Justice marches are taki...","Family, friends, neighbors; We wanted to take ...",2.0,https://www.facebook.com/Linda4Workers/photos/...,https://www.facebook.com/322314941151928/posts...,2926,5.915888,1.0,201,208,33,190,1,0,0,0,0,28,52,23,8,10,2,1,10,1,0,8,2904250,The Coeur d'Alene Tribe,cdatribe,https://scontent-sea1-1.xx.fbcdn.net/v/t31.181...,facebook_page,US,Welcome to the official page for the Schitsu'u...,False,2012-02-08 21:52:13,en,0,2904250|3245155888867804,[Minneapolis],[Canada],[],"[canada, minneapolis]"
4,4,146265275458081_3032237670194146,Facebook,2020-06-05 23:59:26,2021-06-28 20:32:20,status,Black Lives Matter protest in Ada honors Antho...,kxii.com,"Today, No Peace Until Justice marches are taki...",Check out the latest episode of the In the Zon...,2.0,https://www.facebook.com/Linda4Workers/photos/...,https://www.facebook.com/groups/14626527545808...,1188,-22.000000,1.0,0,0,0,0,0,0,0,0,0,0,2,2,3,2,0,2,0,0,0,1,10227323,Y.O.U. Young Opulent & Unique,cdatribe,https://scontent-sea1-1.xx.fbcdn.net/v/t1.1816...,facebook_group,US,Welcome to the official page for the Schitsu'u...,False,2012-02-08 21:52:13,en,0,10227323|3032237670194146,[],[Canada],[],[canada]
...,...,...,...,...,...,...,...,...,...,..

In [41]:
ct_ner['NER-1']=ct_ner['NER']

In [42]:
ct_ner['NER-1'][0]

['ada']

In [43]:
import os
import collections
import ast
import pandas as pd
import numpy as np
from scipy import stats
from datetime import datetime, timedelta
import spacy
import tensorflow as tf
import tensorflow_hub as hub
from tensorflow.keras import layers
#from bert import tokenization
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
import matplotlib.pyplot as plt

In [44]:
ct_ner[:100]

,Unnamed: 0,platformId,platform,date,updated,type1,title,caption,description,message,expandedLinks,link,postUrl,subscriberCount,score,media,actual_likeCount,actual_shareCount,actual_commentCount,actual_loveCount,actual_wowCount,actual_hahaCount,actual_sadCount,actual_angryCount,actual_thankfulCount,actual_careCount,expected_likeCount,expected_shareCount,expected_commentCount,expected_loveCount,expected_wowCount,expected_hahaCount,expected_sadCount,expected_angryCount,expected_thankfulCount,expected_careCount,account_id,account_name,account_handle,account_profileImage,account_accountType,account_pageAdminTopCountry,account_pageDescription,account_verified,account_pageCreatedDate,languageCode,legacyId,post_id,NER-msg,NER-desc,NER-title,NER,NER-1
0,0,48597654813_10158381384639814,Facebook,2020-06-05 23:59:48,2021-08-31 05:39:29,link,Black Lives Matter protest in Ada honors Antho...,kxii.com,-,Hundreds of people came together for a Black L...,1.0,https://www.kxii.com/content/news/Black-Lives-...,https://www.facebook.com/48597654813/posts/101...,125613,-2.015625,1.0,25,11,7,16,0,3,0,2,0,0,24,14,35,5,8,14,16,13,0,4,24653,KXII-TV,channel12,https://scontent-sea1-1.xx.fbcdn.net/v/t1.6435...,facebook_page,US,Texoma's Breaking News & Weather Authority ser...,True,2009-02-04 21:03:03,en,0,24653|10158381384639814,[Ada],[],[],[ada],[ada]
1,1,57406800343_10163759708125344,Facebook,2020-06-05 23:59:38,2021-07-27 09:14:41,status,Black Lives Matter protest in Ada honors Antho...,kxii.com,-,Check out the latest episode of the In the Zon...,1.0,https://www.kxii.com/content/news/Black-Lives-...,https://www.facebook.com/groups/57406800343/pe...,1373,-0.666667,1.0,0,1,0,0,0,0,0,0,0,1,1,1,0,1,0,0,0,0,0,0,8520182,Intertwine Chicago,channel12,https://scontent-sea1-1.xx.fbcdn.net/v/t1.1816...,facebook_group,US,Texoma's Breaking News & Weather Authority ser...,False,2009-02-04 21:03:03,en,0,8520182|10163759708125344,[],[],[],[],[]
2,2,102051553222110_2992368327523737,Facebook,2020-06-05 23:59:37,2021-07-08 21:08:55,photo,Black Lives Matter protest in Ada honors Antho...,kxii.com,"Today, No Peace Until Justice marches are taki...",Read Linda Silas' full statement here: https:/...,2.0,https://www.facebook.com/Linda4Workers/photos/...,https://www.facebook.com/102051553222110/posts...,8927,2.681818,1.0,44,10,0,5,0,0,0,0,0,0,7,5,2,3,1,1,2,1,0,2,190096,Canadian Federation of Nurses Unions (CFNU),NursesUnions,https://scontent-sea1-1.xx.fbcdn.net/v/t1.6435...,facebook_page,CA,The official Facebook page of the Canadian Fed...,False,2011-06-01 18:46:23,en,0,190096|2992368327523737,[],[Canada],[],[canada],[canada]
3,3,322314941151928_3245155888867804,Facebook,2020-06-05 23:59:32,2021-08-24 19:47:41,status,Black Lives Matter protest in Ada honors Antho...,kxii.com,"Today, No Peace Until Justice marches are taki...","Family, friends, neighbors; We wanted to take ...",2.0,https://www.facebook.com/Linda4Workers/photos/...,https://www.facebook.com/322314941151928/posts...,2926,5.915888,1.0,201,208,33,190,1,0,0,0,0,28,52,23,8,10,2,1,10,1,0,8,2904250,The Coeur d'Alene Tribe,cdatribe,https://scontent-sea1-1.xx.fbcdn.net/v/t31.181...,facebook_page,US,Welcome to the official page for the Schitsu'u...,False,2012-02-08 21:52:13,en,0,2904250|3245155888867804,[Minneapolis],[Canada],[],"[canada, minneapolis]","[canada, minneapolis]"
4,4,146265275458081_3032237670194146,Facebook,2020-06-05 23:59:26,2021-06-28 20:32:20,status,Black Lives Matter protest in Ada honors Antho...,kxii.com,"Today, No Peace Until Justice marches are taki...",Check out the latest episode of the In the Zon...,2.0,https://www.facebook.com/Linda4Workers/photos/...,https://www.facebook.com/groups/14626527545808...,1188,-22.000000,1.0,0,0,0,0,0,0,0,0,0,0,2,2,3,2,0,2,0,0,0,1,10227323,Y.O.U. Young Opulent & Unique,cdatribe,https://scontent-sea1-1.xx.fbcdn.net/v/t1.1816...,facebook_group,US,Welcome to the official page for the Schitsu'u...,False,2012-02-08 21:52:13,en,0,10227323|3032237670194146,[],[Canada],[],[ca

In [45]:
%cd mapping_UK

/content/gdrive/My Drive/2021-online-language-offline-impact-main/mapping_UK


In [46]:
!ls

Counties_UK1.txt	       dataframe_mapping_0605.csv
Counties_UK.txt		       dataframe_mapping_0606.csv
Country.txt		       dataframe_mapping_0607.csv
dataframe0529_post_remove.csv  dataframe_mapping_0608.csv
dataframe0530_post_remove.csv  dataframe_mapping_0609.csv
dataframe0531_post_remove.csv  dataframe_mapping_0610.csv
dataframe0601_post_remove.csv  dataframe_mapping_0611.csv
dataframe0602_post_remove.csv  dataframe_mapping_remove_0529.csv
dataframe0603_post_remove.csv  dataframe_mapping_remove_0530.csv
dataframe0604_post_remove.csv  dataframe_mapping_remove_0531.csv
dataframe0607_post_remove.csv  dataframe_mapping_remove_0601.csv
dataframe0608_post_remove.csv  dataframe_mapping_remove_0602.csv
dataframe0609_post_remove.csv  dataframe_mapping_remove_0603.csv
dataframe0610_post_remove.csv  dataframe_mapping_remove_0604.csv
dataframe0611_post_remove.csv  dataframe_mapping_remove_0606.csv
dataframe_mapping	       dataframe_mapping_remove_0607.csv
dataframe_mapping_0529.csv     dataframe

In [47]:
ct_mapping=pd.read_csv('dataframe_mapping_0605.csv',encoding='ISO-8859-1')

In [48]:
ct_mapping

,Unnamed: 0,City,Number,Mapping,Latitude,Longtitude
0,0,minneapolis,7327,"Minneapolis, Hennepin County, Minnesota, 55415...",44.977300,-93.265469
1,1,america,3037,"America, Horst aan de Maas, Limburg, Nederland",51.437148,5.979900
2,2,us,2497,United States,39.783730,-100.445882
3,3,the united states,1633,"The Summit, Guadalupe County, Texas, United St...",29.633008,-97.988594
4,4,minnesota,1442,"Minnesota, United States",45.989659,-94.611329
...,...,...,...,...,...,...
4646,4646,shunpike,1,"Shunpike, Town of Washington, Dutchess County,...",41.828705,-73.708462
4647,4647,seattle city,1,"Seattle, Kaunlaran, Cubao, 4th District, Quezo...",14.619549,121.044016
4648,4648,seattle city of literature,1,NaN,NaN,NaN
4649,4649,the city of lakeland,1,"Lakeland, 2, Wheeler Gate, Lace Market, The Me...",52.952635,-1.150697


In [49]:
country_list=pd.read_csv('Country.txt')
country_list['Name']=country_list['Name'].str.lower()

In [50]:
country_list['Name']

0            afghanistan
1          �land islands
2                albania
3                algeria
4         american samoa
             ...        
250                   us
251    the united states
252              america
253                  u.s
254        united states
Name: Name, Length: 255, dtype: object

In [51]:
states_list=pd.read_csv('states.csv')
states_list['State']

0                             Alabama, United States
1                              Alaska, United States
2                             Arizona, United States
3                            Arkansas, United States
4                          California, United States
                           ...                      
165         Armagh, Northern Ireland, United Kingdom
166           Down, Northern Ireland, United Kingdom
167      Fermanagh, Northern Ireland, United Kingdom
168    Londonderry, Northern Ireland, United Kingdom
169         Tyrone, Northern Ireland, United Kingdom
Name: State, Length: 170, dtype: object

In [52]:
from numpy.core.numeric import NaN
for i in range(len(ct_mapping['City'])):
  for k in range(len(country_list['Name'])):
    if ct_mapping['City'][i]==country_list['Name'][k]:
      ct_mapping['Mapping'][i]=NaN

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [53]:
for i in range(len(ct_mapping['City'])):
  for k in range(len(states_list['State'])):
    if ct_mapping['Mapping'][i]==states_list['State'][k]:
      ct_mapping['Mapping'][i]=NaN

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [54]:
for i in range(len(ct_mapping['City'])):
  for k in range(len(country_list['Name'])):
    if ct_mapping['Mapping'][i]==country_list['Name'][k]:
      ct_mapping['Mapping'][i]=NaN

In [55]:
pd.set_option('display.max_rows', None)

In [56]:
ct_mapping

,Unnamed: 0,City,Number,Mapping,Latitude,Longtitude
0,0,minneapolis,7327,"Minneapolis, Hennepin County, Minnesota, 55415...",44.977300,-93.265469
1,1,america,3037,NaN,51.437148,5.979900
2,2,us,2497,NaN,39.783730,-100.445882
3,3,the united states,1633,NaN,29.633008,-97.988594
4,4,minnesota,1442,NaN,45.989659,-94.611329
5,5,u.s.,1425,NaN,48.684067,9.009582
6,6,washington,1008,"Washington, District of Columbia, United States",38.895037,-77.036543
7,7,texas,854,NaN,31.168570,-99.683001
8,8,new york,752,"New York, United States",40.712728,-74.006015
9,9,chicago,734,"Chicago, Cook County, Illinois, United States",41.875562,-87.624421


In [57]:
ct_mapping

,Unnamed: 0,City,Number,Mapping,Latitude,Longtitude
0,0,minneapolis,7327,"Minneapolis, Hennepin County, Minnesota, 55415...",44.977300,-93.265469
1,1,america,3037,NaN,51.437148,5.979900
2,2,us,2497,NaN,39.783730,-100.445882
3,3,the united states,1633,NaN,29.633008,-97.988594
4,4,minnesota,1442,NaN,45.989659,-94.611329
5,5,u.s.,1425,NaN,48.684067,9.009582
6,6,washington,1008,"Washington, District of Columbia, United States",38.895037,-77.036543
7,7,texas,854,NaN,31.168570,-99.683001
8,8,new york,752,"New York, United States",40.712728,-74.006015
9,9,chicago,734,"Chicago, Cook County, Illinois, United States",41.875562,-87.624421


In [58]:
ct_mapping.to_csv('dataframe_mapping_0605.csv')

In [59]:
#ct_mapping=pd.read_csv('dataframe_mapping_0530.csv',encoding='ISO-8859-1')

In [60]:
for k in range(len(ct_ner['NER-1'])):
  list=[]
  for i in ct_ner['NER-1'][k]:
    for j in range(len(ct_mapping['City'])):
      if(i==ct_mapping['City'][j]):
        list.append(ct_mapping['Mapping'][j])
  ct_ner['NER-1'][k]=list

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


In [61]:
ct_ner[:100]

,Unnamed: 0,platformId,platform,date,updated,type1,title,caption,description,message,expandedLinks,link,postUrl,subscriberCount,score,media,actual_likeCount,actual_shareCount,actual_commentCount,actual_loveCount,actual_wowCount,actual_hahaCount,actual_sadCount,actual_angryCount,actual_thankfulCount,actual_careCount,expected_likeCount,expected_shareCount,expected_commentCount,expected_loveCount,expected_wowCount,expected_hahaCount,expected_sadCount,expected_angryCount,expected_thankfulCount,expected_careCount,account_id,account_name,account_handle,account_profileImage,account_accountType,account_pageAdminTopCountry,account_pageDescription,account_verified,account_pageCreatedDate,languageCode,legacyId,post_id,NER-msg,NER-desc,NER-title,NER,NER-1
0,0,48597654813_10158381384639814,Facebook,2020-06-05 23:59:48,2021-08-31 05:39:29,link,Black Lives Matter protest in Ada honors Antho...,kxii.com,-,Hundreds of people came together for a Black L...,1.0,https://www.kxii.com/content/news/Black-Lives-...,https://www.facebook.com/48597654813/posts/101...,125613,-2.015625,1.0,25,11,7,16,0,3,0,2,0,0,24,14,35,5,8,14,16,13,0,4,24653,KXII-TV,channel12,https://scontent-sea1-1.xx.fbcdn.net/v/t1.6435...,facebook_page,US,Texoma's Breaking News & Weather Authority ser...,True,2009-02-04 21:03:03,en,0,24653|10158381384639814,[Ada],[],[],[ada],"[Adams County, Ohio, United States]"
1,1,57406800343_10163759708125344,Facebook,2020-06-05 23:59:38,2021-07-27 09:14:41,status,Black Lives Matter protest in Ada honors Antho...,kxii.com,-,Check out the latest episode of the In the Zon...,1.0,https://www.kxii.com/content/news/Black-Lives-...,https://www.facebook.com/groups/57406800343/pe...,1373,-0.666667,1.0,0,1,0,0,0,0,0,0,0,1,1,1,0,1,0,0,0,0,0,0,8520182,Intertwine Chicago,channel12,https://scontent-sea1-1.xx.fbcdn.net/v/t1.1816...,facebook_group,US,Texoma's Breaking News & Weather Authority ser...,False,2009-02-04 21:03:03,en,0,8520182|10163759708125344,[],[],[],[],[]
2,2,102051553222110_2992368327523737,Facebook,2020-06-05 23:59:37,2021-07-08 21:08:55,photo,Black Lives Matter protest in Ada honors Antho...,kxii.com,"Today, No Peace Until Justice marches are taki...",Read Linda Silas' full statement here: https:/...,2.0,https://www.facebook.com/Linda4Workers/photos/...,https://www.facebook.com/102051553222110/posts...,8927,2.681818,1.0,44,10,0,5,0,0,0,0,0,0,7,5,2,3,1,1,2,1,0,2,190096,Canadian Federation of Nurses Unions (CFNU),NursesUnions,https://scontent-sea1-1.xx.fbcdn.net/v/t1.6435...,facebook_page,CA,The official Facebook page of the Canadian Fed...,False,2011-06-01 18:46:23,en,0,190096|2992368327523737,[],[Canada],[],[canada],[nan]
3,3,322314941151928_3245155888867804,Facebook,2020-06-05 23:59:32,2021-08-24 19:47:41,status,Black Lives Matter protest in Ada honors Antho...,kxii.com,"Today, No Peace Until Justice marches are taki...","Family, friends, neighbors; We wanted to take ...",2.0,https://www.facebook.com/Linda4Workers/photos/...,https://www.facebook.com/322314941151928/posts...,2926,5.915888,1.0,201,208,33,190,1,0,0,0,0,28,52,23,8,10,2,1,10,1,0,8,2904250,The Coeur d'Alene Tribe,cdatribe,https://scontent-sea1-1.xx.fbcdn.net/v/t31.181...,facebook_page,US,Welcome to the official page for the Schitsu'u...,False,2012-02-08 21:52:13,en,0,2904250|3245155888867804,[Minneapolis],[Canada],[],"[canada, minneapolis]","[nan, Minneapolis, Hennepin County, Minnesota,..."
4,4,146265275458081_3032237670194146,Facebook,2020-06-05 23:59:26,2021-06-28 20:32:20,status,Black Lives Matter protest in Ada honors Antho...,kxii.com,"Today, No Peace Until Justice marches are taki...",Check out the latest episode of the In the Zon...,2.0,https://www.facebook.com/Linda4Workers/photos/...,https://www.facebook.com/groups/14626527545808...,1188,-22.000000,1.0,0,0,0,0,0,0,0,0,0,0,2,2,3,2,0,2,0,0,0,1,10227323,Y.O.U. Young Opulent & Unique,cdatribe,https://scontent-sea1-1.xx.fbcdn.net/v/t1.1816...,facebook_group,US,Welcome to the official page for the Schitsu'u...,False,2012-02-08 2

In [62]:
ct_ner['NER-1'][:100]

0                   [Adams County, Ohio, United States]
1                                                    []
2                                                 [nan]
3     [nan, Minneapolis, Hennepin County, Minnesota,...
4                                                 [nan]
5     [Covid, Sant'Antonio (Piasco), Piasco, Cuneo, ...
6                                                 [nan]
7     [Denver, Denver County, Colorado, United State...
8     [nan, Minneapolis, Hennepin County, Minnesota,...
9     [Santa Barbara, Santa Barbara County, Californ...
10    [Denver, Denver County, Colorado, United State...
11    [nan, Denver, Denver County, Colorado, United ...
12    [nan, Denver, Denver County, Colorado, United ...
13    [Annapolis, Anne Arundel County, Maryland, Uni...
14    [nan, Minneapolis, Hennepin County, Minnesota,...
15                                                [nan]
16    [nan, Arlington, Tarrant County, Texas, United...
17    [nan, Minneapolis, Hennepin County, Minnes

In [63]:
ct_ner.to_csv('dataframe0605_post_remove.csv')

In [64]:
!ls

Counties_UK1.txt	       dataframe_mapping_0604.csv
Counties_UK.txt		       dataframe_mapping_0605.csv
Country.txt		       dataframe_mapping_0606.csv
dataframe0529_post_remove.csv  dataframe_mapping_0607.csv
dataframe0530_post_remove.csv  dataframe_mapping_0608.csv
dataframe0531_post_remove.csv  dataframe_mapping_0609.csv
dataframe0601_post_remove.csv  dataframe_mapping_0610.csv
dataframe0602_post_remove.csv  dataframe_mapping_0611.csv
dataframe0603_post_remove.csv  dataframe_mapping_remove_0529.csv
dataframe0604_post_remove.csv  dataframe_mapping_remove_0530.csv
dataframe0605_post_remove.csv  dataframe_mapping_remove_0531.csv
dataframe0607_post_remove.csv  dataframe_mapping_remove_0601.csv
dataframe0608_post_remove.csv  dataframe_mapping_remove_0602.csv
dataframe0609_post_remove.csv  dataframe_mapping_remove_0603.csv
dataframe0610_post_remove.csv  dataframe_mapping_remove_0604.csv
dataframe0611_post_remove.csv  dataframe_mapping_remove_0606.csv
dataframe_mapping	       dataframe_mappin